In [ ]:
import json
import os
import zipfile
import numpy as np
from collections import defaultdict
from scipy.ndimage import label
from sklearn.cluster import KMeans

class ARCSolutionGenerator:
    def __init__(self):
        self.pattern_handlers = [
            self.handle_outline_only,          # 1 轮廓留存
            self.handle_center_object,         # 2 对齐中心
            self.handle_diagonal_mirror,       # 3 主对角镜像
            self.handle_color_palette_row,     # 4 颜色调色板
            self.handle_bounding_fill_by_color,# 5 每色填自身 bbox
            self.handle_row_col_propagation,   # 6 行/列扩散
            # 🆕 新增 8 个高频模式
            self.handle_repeat_pattern,         # 1
            self.handle_checkerboard,           # 2
            self.handle_vertical_stripes,       # 3
            self.handle_horizontal_stripes,     # 4
            self.handle_majority_color_fill,    # 5
            self.handle_cross_lines,            # 6
            self.handle_color_swap,             # 7
            self.handle_crop_center,            # 8
            self.handle_single_color_output,     # 🔹NEW (最简单，放最前)
            self.handle_bounding_crop,           # 🔹NEW
            self.handle_overlay_fill,            # 🔹NEW
            self.handle_remove_color,            # 🔹NEW
            self.handle_color_mapping,
            self.handle_grid_operations,
            self.handle_resize,                # ✅ NEW: Resize handler
            self.handle_object_operations,
            self.handle_pattern_replication,
            self.handle_mirror_symmetry,
            self.handle_arithmetic_operations,
            self.handle_conditional_operations,
            # 🆕 新增的 10 个高命中
            self.handle_upscale_nearest,            # 1 最近邻放大 (k×)
            self.handle_downscale_sample,           # 2 等间隔抽样缩小
            self.handle_fill_gaps_rowwise,          # 3 行内补洞（同色端点连线）
            self.handle_background_to_majority,     # 4 背景改为主色
            self.handle_keep_dominant_color,        # 5 仅保留最常见颜色
            self.handle_extract_first_nonempty_rowcol, # 6 抽第一条非空行/列
            self.handle_sort_rows_by_density,       # 7 按非零密度排序行
            self.handle_sort_cols_by_density,       # 8 按非零密度排序列
            self.handle_complete_by_mirror_half,    # 9 半幅镜像补全
            self.handle_palette_column,             # 10 竖列调色板
            self.handle_uniform_row_fill,       # A 行内统一成主色
            self.handle_uniform_col_fill,       # B 列内统一成主色
            self.handle_draw_frame,             # C 绘制边框
            self.handle_main_diag_line,         # D 主对角线涂色
            self.handle_anti_diag_line,         # E 副对角线涂色
            self.handle_border_to_zero,         # F 抹掉四周 1px 边
            self.handle_single_object_translate,# G 按矢量平移单物体
            self.handle_object_count_row,       # H 输出计数条
            self.handle_color_shift_plus_one,   # I 所有颜色+1 mod 10
            self.handle_keep_colors,            # J 仅保留训练示例中的颜色
            self.handle_remove_small_objects,   # K 删除极小噪点
            self.handle_duplicate_quadrant,     # L 复制象限到整图
        ]

    # A 行内主色填充：若每行最多两色且输出行全同色
    def handle_uniform_row_fill(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        ok=True
        for r1,r2 in zip(a,b):
            maj = np.bincount(r1).argmax()
            if not np.all(r2==maj): ok=False; break
        if not ok: return None
        return """def p(g):
 return [[max(set(r), key=r.count)]*len(r) for r in g]
"""

    # B 列内主色填充
    def handle_uniform_col_fill(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        H,W=a.shape
        ok=True
        for j in range(W):
            col=a[:,j].tolist()
            maj=max(set(col), key=col.count)
            if not np.all(b[:,j]==maj): ok=False; break
        if not ok: return None
        return """def p(g):
 H,W=len(g),len(g[0])
 cols=[[g[i][j] for i in range(H)] for j in range(W)]
 res=[[0]*W for _ in range(H)]
 for j,col in enumerate(cols):
  maj=max(set(col), key=col.count)
  for i in range(H): res[i][j]=maj
 return res
"""

    # C 绘制边框：若输出是在输入外围加 1px 实色框
    def handle_draw_frame(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if b.shape[0]!=a.shape[0]+2 or b.shape[1]!=a.shape[1]+2: return None
        frame_color = int(b[0,0])
        inner = b[1:-1,1:-1]
        if np.array_equal(inner,a) and \
           np.all(b[[0,-1],:]==frame_color) and np.all(b[:,[0,-1]]==frame_color):
            return f"""def p(g):
 H,W=len(g),len(g[0])
 c={frame_color}
 out=[[c]*(W+2)]
 for r in g: out.append([c]+r+[c])
 out.append([c]*(W+2))
 return out
"""
        return None

    # D 主对角线
    def handle_main_diag_line(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        H,W=a.shape
        if H!=W: return None
        diff = np.where(a!=b)
        if len(diff[0])==H and all(i==j for i,j in zip(*diff)):
            c=int(b[0,0])
            return f"""def p(g):
 n=len(g)
 return [[ {c} if i==j else g[i][j] for j in range(n)] for i in range(n)]
"""
        return None

    # E 副对角线
    def handle_anti_diag_line(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        H,W=a.shape
        if H!=W: return None
        diff = np.where(a!=b)
        if len(diff[0])==H and all(i+j==H-1 for i,j in zip(*diff)):
            c=int(b[0,W-1])
            return f"""def p(g):
 n=len(g)
 return [[ {c} if i+j==n-1 else g[i][j] for j in range(n)] for i in range(n)]
"""
        return None

    # F 抹掉四周 1px
    def handle_border_to_zero(self, task_data, analysis):
        a,b = map(np.array,(task_data['train'][0]['input'],
                            task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        inner = a[1:-1,1:-1]
        if np.array_equal(b[1:-1,1:-1], inner) and np.all(b[[0,-1],:]==0) \
           and np.all(b[:,[0,-1]]==0):
            return """def p(g):
 H,W=len(g),len(g[0])
 out=[r[:] for r in g]
 for i in range(H):
  out[i][0]=out[i][-1]=0
 for j in range(W):
  out[0][j]=out[-1][j]=0
 return out
"""
        return None

    # ---------- D. single_object_translate ----------
    def handle_single_object_translate(self, task_data, analysis):
        props = analysis['object_properties'][0][0]
        mv = props['position_change']
        if isinstance(mv, str):
            return None
        dy, dx = map(int, mv)
        if (dy, dx) == (0, 0):
            return None
        return f'''def p(g):
 H, W = len(g), len(g[0])
 ys = [i for i,row in enumerate(g) for v in row if v]
 xs = [j for i,row in enumerate(g) for j,v in enumerate(row) if v]
 res = [[0]*W for _ in range(H)]
 for i,j in zip(ys, xs):
     ni, nj = i+{dy}, j+{dx}
     if 0 <= ni < H and 0 <= nj < W:
         res[ni][nj] = g[i][j]
 return res
'''

    # H 计数条：输出 1×n, n 为某色计数
    def handle_object_count_row(self, task_data, analysis):
        a,b=(task_data['train'][0]['input'],task_data['train'][0]['output'])
        if len(b)!=1: return None
        flat=[x for r in a for x in r if x!=0]
        if not flat: return None
        maj=max(set(flat), key=flat.count)
        cnt=flat.count(maj)
        if b==[[maj]*cnt]:
            return f"""def p(g):
 flat=[x for r in g for x in r if x!=0]
 maj=max(set(flat), key=flat.count)
 return [[maj]*flat.count(maj)]
"""
        return None

    # I +1 mod 10
    def handle_color_shift_plus_one(self, task_data, analysis):
        a,b=(task_data['train'][0]['input'],task_data['train'][0]['output'])
        if [[(x+1)%10 for x in r] for r in a]==b:
            return """def p(g):
 return [[(x+1)%10 for x in r] for r in g]
"""
        return None

    # J 仅保留训练中用到的颜色，其余置零
    def handle_keep_colors(self, task_data, analysis):
        keep=set(x for ex in task_data['train']
                   for r in ex['output'] for x in r)
        if len(keep)>=10: return None
        a,b=(task_data['train'][0]['input'],task_data['train'][0]['output'])
        if [[x if x in keep else 0 for x in r] for r in a]==b:
            keep_str=",".join(map(str,keep))
            return f"""def p(g):
 keep={{ {keep_str} }}
 return [[x if x in keep else 0 for x in r] for r in g]
"""
        return None

    # K 删除小物体（≤2 像素）
    def handle_remove_small_objects(self, task_data, analysis):
        a,b=map(np.array,(task_data['train'][0]['input'],
                          task_data['train'][0]['output']))
        if a.shape!=b.shape: return None
        diff=a!=b
        if not diff.any(): return None
        ys,xs=np.where(diff)
        if len(ys)<=2 and np.all(b[ys,xs]==0):
            coords=[(int(y),int(x)) for y,x in zip(ys,xs)]
            return f"""def p(g):
 g=[r[:] for r in g]
 for y,x in {coords}: g[y][x]=0
 return g
"""
        return None

    # L 将左上象限复制到 4 象限
    def handle_duplicate_quadrant(self, task_data, analysis):
        a,b=map(np.array,(task_data['train'][0]['input'],
                          task_data['train'][0]['output']))
        h,w=a.shape
        if h%2 or w%2 or b.shape!=(h,w): return None
        q=a[:h//2,:w//2]
        cand=np.block([[q,q],[q,q]])
        if np.array_equal(cand,b):
            return """def p(g):
 h,w=len(g),len(g[0]);hh,ww=h//2,w//2
 q=[row[:ww] for row in g[:hh]]
 return [q_row+q_row for q_row in q]+[q_row+q_row for q_row in q]
"""
        return None
    # ───────── 1) 最近邻放大 (整数 kx, ky) ─────────
    def handle_upscale_nearest(self, task_data, analysis):
        a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
        hi, wi = a.shape; ho, wo = b.shape
        if ho%hi or wo%wi: return None
        ky, kx = ho//hi, wo//wi
        ok = True
        for i in range(hi):
            for j in range(wi):
                block = b[i*ky:(i+1)*ky, j*kx:(j+1)*kx]
                if not np.all(block == a[i,j]): ok=False; break
            if not ok: break
        if not ok: return None
        return f"""def p(g):
 ky, kx = {ky}, {kx}
 H, W = len(g), len(g[0])
 return [[ g[i//ky][j//kx] for j in range(W*kx) ] for i in range(H*ky)]
"""

    # ───────── 2) 等间隔抽样缩小 (取每 ky,kx 个像素) ─────────
    def handle_downscale_sample(self, task_data, analysis):
        a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
        hi, wi = a.shape; ho, wo = b.shape
        if hi%ho or wi%wo: return None
        ky, kx = hi//ho, wi//wo
        if not np.all(b == a[::ky, ::kx]): return None
        return f"""def p(g):
 ky, kx = {ky}, {kx}
 return [[ g[i*ky][j*kx] for j in range(len(g[0])//kx) ] for i in range(len(g)//ky)]
"""

    # ───────── 3) 行内补洞（同色端点连线）─────────
    def handle_fill_gaps_rowwise(self, task_data, analysis):
        a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
        if a.shape != b.shape: return None
        def fill_row(r):
            r = r[:]
            for c in set(x for x in r if x!=0):
                idx = [j for j,x in enumerate(r) if x==c]
                if idx:
                    l, rgt = min(idx), max(idx)
                    for j in range(l, rgt+1):
                        if r[j]==0: r[j]=c
            return r
        cand = np.array([fill_row(list(row)) for row in a.tolist()])
        if not np.array_equal(cand, b): return None
        return """def p(g):
 res=[]
 for r in g:
  r=r[:]
  s={x for x in r if x!=0}
  for c in s:
   idx=[j for j,x in enumerate(r) if x==c]
   l,rgt=min(idx),max(idx)
   for j in range(l,rgt+1):
    if r[j]==0: r[j]=c
  res.append(r)
 return res
"""

    # ───────── 4) 背景改为主色（动态计算多数色，保留前景）─────────
    def handle_background_to_majority(self, task_data, analysis):
        a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        # 运行时计算主色：排除 0（常作背景）
        def apply(g):
            flat=[x for r in g for x in r if x!=0]
            c=max(range(10), key=lambda v: flat.count(v)) if flat else 0
            return [[x if x!=0 else c for x in r] for r in g]
        if apply(a)==b:
            return """def p(g):
 flat=[x for r in g for x in r if x!=0]
 c=max(range(10), key=lambda v: flat.count(v)) if flat else 0
 return [[x if x!=0 else c for x in r] for r in g]
"""
        return None

    # ───────── 5) 仅保留最常见颜色（非零）─────────
    def handle_keep_dominant_color(self, task_data, analysis):
        a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        def apply(g):
            flat=[x for r in g for x in r if x!=0]
            if not flat: return [[0 for _ in r] for r in g]
            c=max(range(10), key=lambda v: flat.count(v))
            return [[x if x==c else 0 for x in r] for r in g]
        if apply(a)==b:
            return """def p(g):
 flat=[x for r in g for x in r if x!=0]
 if not flat: return [[0 for _ in r] for r in g]
 c=max(range(10), key=lambda v: flat.count(v))
 return [[x if x==c else 0 for x in r] for r in g]
"""
        return None

    # ───────── 6) 抽第一条非空行/列（变成 1×W 或 H×1）─────────
    def handle_extract_first_nonempty_rowcol(self, task_data, analysis):
        a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        # 行
        for i, r in enumerate(a):
            if any(x!=0 for x in r):
                if b == [r]:
                    return """def p(g):
 for r in g:
  if any(x!=0 for x in r): return [r]
 return [g[0]]
"""
                break
        # 列
        H, W = len(a), len(a[0])
        col = None
        for j in range(W):
            c = [a[i][j] for i in range(H)]
            if any(x!=0 for x in c):
                col = [[x] for x in c]; break
        if col and b == col:
            return """def p(g):
 H,W=len(g),len(g[0])
 for j in range(W):
  c=[g[i][j] for i in range(H)]
  if any(x!=0 for x in c): return [[x] for x in c]
 return [[g[i][0]] for i in range(H)]
"""
        return None

    # ───────── 7) 行按非零密度排序（升或降）─────────
    def handle_sort_rows_by_density(self, task_data, analysis):
        a, b = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        if len(a)!=len(b) or len(a[0])!=len(b[0]): return None
        asc  = sorted(a, key=lambda r: sum(x!=0 for x in r))
        desc = asc[::-1]
        if b==asc:
            return """def p(g):
 return sorted(g, key=lambda r: sum(x!=0 for x in r))
"""
        if b==desc:
            return """def p(g):
 return sorted(g, key=lambda r: sum(x!=0 for x in r), reverse=True)
"""
        return None

    # ───────── 8) 列按非零密度排序（升或降）─────────
    def handle_sort_cols_by_density(self, task_data, analysis):
        a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
        if a.shape != b.shape: return None
        H,W=a.shape
        def sort_cols(arr, rev=False):
            cols=[arr[:,j].tolist() for j in range(W)]
            cols=sorted(cols, key=lambda c: sum(x!=0 for x in c), reverse=rev)
            # 还原为行列表
            return [[cols[j][i] for j in range(W)] for i in range(H)]
        if np.array_equal(b, np.array(sort_cols(a, False))):
            return """def p(g):
 H,W=len(g),len(g[0])
 cols=[[g[i][j] for i in range(H)] for j in range(W)]
 cols=sorted(cols, key=lambda c: sum(x!=0 for x in c))
 return [[cols[j][i] for j in range(W)] for i in range(H)]
"""
        if np.array_equal(b, np.array(sort_cols(a, True))):
            return """def p(g):
 H,W=len(g),len(g[0])
 cols=[[g[i][j] for i in range(H)] for j in range(W)]
 cols=sorted(cols, key=lambda c: sum(x!=0 for x in c), reverse=True)
 return [[cols[j][i] for j in range(W)] for i in range(H)]
"""
        return None

    # ───────── 9) 半幅镜像补全（左右或上下）─────────
    def handle_complete_by_mirror_half(self, task_data, analysis):
        a, b = map(np.array, (task_data['train'][0]['input'], task_data['train'][0]['output']))
        if a.shape != b.shape: return None
        H,W=a.shape
        # 左→右镜像
        mid=W//2
        left=a[:, :mid]
        cand_lr=np.hstack([left, np.fliplr(left)])
        if np.array_equal(b, cand_lr):
            return """def p(g):
 H,W=len(g),len(g[0]);m=W//2
 left=[r[:m] for r in g]
 return [ left[i]+left[i][::-1] for i in range(H) ]
"""
        # 上→下镜像
        mid=H//2
        top=a[:mid, :]
        cand_ud=np.vstack([top, np.flipud(top)])
        if np.array_equal(b, cand_ud):
            return """def p(g):
 H,W=len(g),len(g[0]);m=H//2
 top=g[:m]
 return top + top[::-1]
"""
        return None

    # ───────── 10) 竖列调色板（按升序列出出现颜色）─────────
    def handle_palette_column(self, task_data, analysis):
        inp, out = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        colors = sorted({x for r in inp for x in r})
        if out == [[c] for c in colors]:
            return """def p(g):
 c=sorted({x for r in g for x in r})
 return [[x] for x in c]
"""
        return None
        
    # ────────── 1. 整体重复平铺 ──────────
    def handle_repeat_pattern(self, task_data, analysis):
        inp = task_data['train'][0]['input']
        out = task_data['train'][0]['output']
        ry = len(out) // len(inp)
        rx = len(out[0]) // len(inp[0])
        tiled = [[inp[i % len(inp)][j % len(inp[0])]
                  for j in range(len(inp[0]) * rx)]
                 for i in range(len(inp) * ry)]
        if tiled == out:
            return f"""def p(g):
 ry, rx = {ry}, {rx}
 h, w = len(g), len(g[0])
 return [[g[i % h][j % w] for j in range(w * rx)]
         for i in range(h * ry)]
"""

    # ────────── 2. 棋盘格涂色 ──────────
    def handle_checkerboard(self, task_data, analysis):
        """黑白棋盘格常见：0/1 或两色交替"""
        out = np.array(task_data['train'][0]['output'])
        if np.all(out==out[0,0] ^ ((np.indices(out.shape).sum(0))%2)):
            base=int(out[0,0]); other=int(list(set(out.ravel()))[1])
            return f"""def p(g):
 return[[({base} if (i+j)%2==0 else {other}) for j in range(len(g[0]))] for i in range(len(g))]
"""
        return None

    # ────────── 3. 垂直条纹 ──────────
    def handle_vertical_stripes(self, task_data, analysis):
        """整列统一成某色"""
        inp,out=(np.array(task_data['train'][0][k]) for k in('input','output'))
        if out.shape!=inp.shape: return None
        for j in range(out.shape[1]):
            if len(set(out[:,j]))==1 and not np.array_equal(inp[:,j],out[:,j]):
                return f"""def p(g):
 H,W=len(g),len(g[0])
 return[[g[i][0] for _ in range(W)] for i in range(H)]
"""
        return None

    # ────────── 4. 水平条纹 ──────────
    def handle_horizontal_stripes(self, task_data, analysis):
        """整行统一成某色"""
        inp,out=(np.array(task_data['train'][0][k]) for k in('input','output'))
        if out.shape!=inp.shape: return None
        for i in range(out.shape[0]):
            if len(set(out[i]))==1 and not np.array_equal(inp[i],out[i]):
                return """def p(g):
 return[[x for _ in g[0]] for x in [r[0] for r in g]]
"""
        return None

    # ────────── 5. 主色填充 ──────────
    def handle_majority_color_fill(self, task_data, analysis):
        """输出全填输入里出现次数最多的颜色"""
        inp, out=(task_data['train'][0]['input'],task_data['train'][0]['output'])
        flat=[x for r in inp for x in r]
        maj=max(set(flat), key=flat.count)
        if all(all(x==maj for x in r) for r in out):
            return f"""def p(g):
 from collections import Counter
 flat=[x for r in g for x in r]
 maj=Counter(flat).most_common(1)[0][0]
 return[[maj]*len(g[0]) for _ in g]
"""
        return None

    # ────────── 6. 中心十字线 ──────────
    def handle_cross_lines(self, task_data, analysis):
        """在中心画十字线"""
        inp,out=(np.array(task_data['train'][0][k]) for k in('input','output'))
        if out.shape!=inp.shape: return None
        H,W=out.shape; cy,cx=H//2,W//2
        if np.all(out[cy,:]==out[cy,0]) and np.all(out[:,cx]==out[0,cx]):
            c=int(out[cy,cx])
            return f"""def p(g):
 H,W=len(g),len(g[0]);cy,cx=H//2,W//2
 return[[{c} if i==cy or j==cx else g[i][j] for j in range(W)] for i in range(H)]
"""
        return None

    # ────────── 7. 两色互换 ──────────
    def handle_color_swap(self, task_data, analysis):
        """仅两种色互换 (a↔b)"""
        inp,out=(task_data['train'][0]['input'],task_data['train'][0]['output'])
        in_set={x for r in inp for x in r}; out_set={x for r in out for x in r}
        if len(in_set)==len(out_set)==2 and in_set==out_set:
            a,b=in_set
            return f"""def p(g):
 return[[{a} if x=={b} else ({b} if x=={a} else x) for x in r] for r in g]
"""
        return None

    # ────────── 8. 裁剪中心区域 ──────────
    def handle_crop_center(self, task_data, analysis):
        """输出=输入的正中间子矩阵"""
        inp,out=(np.array(task_data['train'][0][k]) for k in('input','output'))
        h,w=inp.shape; ho,wo=out.shape
        cy,cx=(h-ho)//2,(w-wo)//2
        if np.array_equal(out, inp[cy:cy+ho, cx:cx+wo]):
            return f"""def p(g):
 h,w=len(g),len(g[0]);ho,wo={ho},{wo}
 cy,cx=(h-ho)//2,(w-wo)//2
 return[g[i][cx:cx+wo] for i in range(cy,cy+ho)]
"""
        return None
    # ───────────────── 1. 轮廓留存 ─────────────────
    def handle_outline_only(self, task_data, analysis):
        """若输出仅保留输入物体外边缘像素，其余清零"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        h, w = inp.shape
        # 4-邻域检测边缘
        edge = np.zeros_like(inp)
        for i in range(h):
            for j in range(w):
                if inp[i,j]==0: continue
                nbor = [(i+1,j),(i-1,j),(i,j+1),(i,j-1)]
                if any(not(0<=y<h and 0<=x<w) or inp[y,x]==0 for y,x in nbor):
                    edge[i,j] = inp[i,j]
        if np.array_equal(out, edge):
            return """def p(g):
 h,w=len(g),len(g[0])
 return[[ g[i][j] if g[i][j] and any( (y<0 or y>=h or x<0 or x>=w or g[y][x]==0)
        for y,x in ((i+1,j),(i-1,j),(i,j+1),(i,j-1)) )
        else 0 for j in range(w)] for i in range(h)]
"""
        return None

    # ───────────────── 2. 把物体搬到中心 ─────────────────
    # ---------- B. center_object (移除 np.array) ----------
    def handle_center_object(self, task_data, analysis):
        inp = task_data['train'][0]['input']
        out = task_data['train'][0]['output']
        if len(inp) != len(out) or len(inp[0]) != len(out[0]):
            return None
        ys = [i for i, row in enumerate(inp) for v in row if v]
        xs = [j for row in inp for j, v in enumerate(row) if v]
        if not ys:
            return None
        y0, y1 = min(ys), max(ys) + 1
        x0, x1 = min(xs), max(xs) + 1
        crop = [r[x0:x1] for r in inp[y0:y1]]
        H, W = len(inp), len(inp[0])
        h, w = len(crop), len(crop[0])
        cy, cx = (H - h) // 2, (W - w) // 2
        canvas = [[0] * W for _ in range(H)]
        for i in range(h):
            canvas[cy + i][cx:cx + w] = crop[i][:]
        if canvas == out:
            return f'''def p(g):
 H, W = len(g), len(g[0])
 ys = [i for i,row in enumerate(g) for v in row if v]
 xs = [j for row in g for j,v in enumerate(row) if v]
 y0, y1 = min(ys), max(ys)+1
 x0, x1 = min(xs), max(xs)+1
 crop = [r[x0:x1] for r in g[y0:y1]]
 h, w = len(crop), len(crop[0])
 cy, cx = (H-h)//2, (W-w)//2
 res = [[0]*W for _ in range(H)]
 for i in range(h):
     res[cy+i][cx:cx+w] = crop[i][:]
 return res
'''

    # ───────────────── 3. 主对角镜像 ─────────────────
    def handle_diagonal_mirror(self, task_data, analysis):
        """输出 = 输入在主对角线镜像 (转置)"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape[0]!=inp.shape[1]: return None  # 需方阵
        if np.array_equal(out, inp.T):
            return """def p(g):
 return[list(r) for r in zip(*g)]
"""
        return None

    # ───────────────── 4. 调色板行 ─────────────────
    def handle_color_palette_row(self, task_data, analysis):
        """输出是一行，按升序列出输入中出现过的颜色"""
        inp, out = (task_data['train'][0]['input'], task_data['train'][0]['output'])
        colors = sorted({x for row in inp for x in row})
        if len(out)==1 and out[0]==colors:
            return """def p(g):
 c=sorted({x for r in g for x in r})
 return [c]
"""
        return None

    # ---------- C. bounding_fill_by_color ----------
    def handle_bounding_fill_by_color(self, task_data, analysis):
        inp = task_data['train'][0]['input']
        out = task_data['train'][0]['output']
        if len(inp) != len(out) or len(inp[0]) != len(out[0]):
            return None
        H, W = len(inp), len(inp[0])
        res = [[0] * W for _ in range(H)]
        colors = {v for row in inp for v in row if v}
        for c in colors:
            ys = [i for i, row in enumerate(inp) for v in row if v == c]
            xs = [j for i, row in enumerate(inp) for j, v in enumerate(row) if v == c]
            y0, y1 = min(ys), max(ys) + 1
            x0, x1 = min(xs), max(xs) + 1
            for i in range(y0, y1):
                for j in range(x0, x1):
                    res[i][j] = c
        if res == out:
            return '''def p(g):
 H, W = len(g), len(g[0])
 res = [[0]*W for _ in range(H)]
 colors = {v for row in g for v in row if v}
 for c in colors:
     ys = [i for i,row in enumerate(g) for v in row if v==c]
     xs = [j for i,row in enumerate(g) for j,v in enumerate(row) if v==c]
     y0,y1 = min(ys), max(ys)+1
     x0,x1 = min(xs), max(xs)+1
     for i in range(y0,y1):
         for j in range(x0,x1):
             res[i][j] = c
 return res
'''

    # ───────────────── 6. 行/列颜色扩散 ─────────────────
    def handle_row_col_propagation(self, task_data, analysis):
        """
        典型: 若一行/列含目标色, 该色向整个行/列扩散
        条件: 输出与输入 shape 相同; 每行(列)要么全原色要么全变某色
        """
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape!=out.shape: return None
        # 行扩散
        if all((np.unique(o).size==1) or np.array_equal(o, i)
               for i,o in zip(inp, out)):
            return """def p(g):
 return[[g[r][0] if len(set(g[r]))>1 else g[r][c]
         for c in range(len(g[0]))] for r in range(len(g))]
"""
        # 列扩散
        if all((np.unique(out[:,c]).size==1) or np.array_equal(out[:,c], inp[:,c])
               for c in range(inp.shape[1])):
            return """def p(g):
 H,W=len(g),len(g[0])
 cols=[[g[r][c] for r in range(H)] for c in range(W)]
 for c in range(W):
     if len(set(cols[c]))>1:
         cols[c]=[cols[c][0]]*H
 return[[cols[c][r] for c in range(W)] for r in range(H)]
"""
        return None
    def analyze_task(self, task_data):
        all_examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        analysis = {
            'color_changes': defaultdict(set),
            'shape_changes': set(),
            'object_properties': [],
            'symmetry': None,
            'arithmetic': None
        }
        
        for example in all_examples:
            in_grid = np.array(example['input'])
            out_grid = np.array(example['output'])
            
            analysis['shape_changes'].add((in_grid.shape, out_grid.shape))
            
            for (i,j), val in np.ndenumerate(in_grid):
                if i < out_grid.shape[0] and j < out_grid.shape[1]:
                    if in_grid[i,j] != out_grid[i,j]:
                        analysis['color_changes'][(i,j)].add((int(in_grid[i,j]), int(out_grid[i,j])))
            
            analysis['object_properties'].append(self.analyze_objects(in_grid, out_grid))
            
            if analysis['symmetry'] is None:
                analysis['symmetry'] = self.detect_symmetry(in_grid, out_grid)
            
            if analysis['arithmetic'] is None:
                analysis['arithmetic'] = self.detect_arithmetic(in_grid, out_grid)
        
        return analysis
    
    def analyze_objects(self, in_grid, out_grid):
        structure = np.ones((3,3), dtype=int)
        labeled_in, n_in = label(in_grid > 0, structure)
        labeled_out, n_out = label(out_grid > 0, structure)
        
        objects = []
        for i in range(1, n_in+1):
            in_obj = (labeled_in == i)
            out_obj = None
            if i <= n_out:
                out_obj = (labeled_out == i)
            objects.append({
                'in_size': int(in_obj.sum()),
                'out_size': int(out_obj.sum()) if out_obj is not None else 0,
                'position_change': self.detect_position_change(in_obj, out_obj)
            })
        return objects
    
    def detect_position_change(self, in_obj, out_obj):
        if out_obj is None:
            return "removed"
        in_pos = np.argwhere(in_obj).mean(axis=0)
        out_pos = np.argwhere(out_obj).mean(axis=0)
        return out_pos - in_pos
    
    def detect_symmetry(self, in_grid, out_grid):
        if np.array_equal(out_grid, np.rot90(in_grid, 1)):
            return 'rotate_90'
        if np.array_equal(out_grid, np.rot90(in_grid, 2)):
            return 'rotate_180'
        if np.array_equal(out_grid, np.rot90(in_grid, 3)):
            return 'rotate_270'
        if np.array_equal(out_grid, np.flipud(in_grid)):
            return 'flip_vertical'
        if np.array_equal(out_grid, np.fliplr(in_grid)):
            return 'flip_horizontal'
        return None
    
    def detect_arithmetic(self, in_grid, out_grid):
        if in_grid.shape != out_grid.shape:
            return None  # ✅ FIX: Skip if shapes differ
            
        diff = out_grid - in_grid
        if np.all(diff == diff[0,0]):
            return ('add', int(diff[0,0]))
        
        with np.errstate(divide='ignore', invalid='ignore'):
            safe_in = np.where(in_grid==0, 1, in_grid)
            ratio = out_grid / safe_in
            if np.all(ratio[in_grid > 0] == ratio[in_grid > 0][0]):
                return ('multiply', float(ratio[in_grid > 0][0]))
        return None
    
    def generate_solution(self, task_data):
        analysis = self.analyze_task(task_data)
        for handler in self.pattern_handlers:
            solution = handler(task_data, analysis)
            if solution and self.verify_solution(solution, task_data):
                return solution
        
        # ✅ Better fallback (identity instead of all zeros)
        return """def p(g):
 return [row[:] for row in g]
"""
    # ---------- 1. 纯色输出 ----------
    def handle_single_color_output(self, task_data, analysis):
        cands = {int(x) for ex in task_data['train']
                       for row in ex['output'] for x in row}
        if len(cands) == 1:
            c = cands.pop()
            return f"""def p(g):
 return[[{c}]*len(g[0]) for _ in g]
"""
        return None

    # ---------- 2. 裁掉空背景 ----------
    def handle_bounding_crop(self, task_data, analysis):
        inp, out = map(np.array, (task_data['train'][0]['input'],
                                  task_data['train'][0]['output']))
        # 找出非 0 像素的最小外接矩形
        nz = np.argwhere(inp)
        if not len(nz):
            return None
        (y0,x0),(y1,x1) = nz.min(0), nz.max(0)+1
        if np.array_equal(out, inp[y0:y1, x0:x1]):
            return f"""def p(g):
 c=[(i,j)for i,r in enumerate(g)for j,x in enumerate(r)if x]
 y0=min(i for i,_ in c); y1=max(i for i,_ in c)+1
 x0=min(j for _,j in c); x1=max(j for _,j in c)+1
 return[g[i][x0:x1] for i in range(y0,y1)]
"""
        return None

    # ---------- 3. 用输入覆盖输出空位 ----------
    def handle_overlay_fill(self, task_data, analysis):
        inp, out = map(np.array, (task_data['train'][0]['input'],
                                  task_data['train'][0]['output']))
        if inp.shape != out.shape:
            return None
        # 输出 = 如果 out==0 用 in 否则保持
        if np.all(np.where(out==0, inp, out) == out):
            return """def p(g):
 return[[g[i][j] if r[j]==0 else r[j]
         for j in range(len(r))]
        for i,r in enumerate(g)]
"""  # ← r = out? No; see below in verify. 需要 run-time access – 用两遍 g 不行
        return None  # （见说明，下方给出可工作版本）

    # ---------- 4. 移除某单一颜色 ----------
    def handle_remove_color(self, task_data, analysis):
        inp, out = map(np.array, (task_data['train'][0]['input'],
                                  task_data['train'][0]['output']))
        if inp.shape != out.shape:
            return None
        diff = inp != out
        # 全部差异来自同一种颜色被清零
        removed = {int(inp[i,j]) for (i,j) in zip(*np.where(diff))}
        if len(removed)==1 and np.all(out[diff]==0):
            c = removed.pop()
            return f"""def p(g):
 return[[0 if x=={c} else x for x in r] for r in g]
"""
        return None
        
    def handle_color_mapping(self, task_data, analysis):
        color_map = {}
        for pos, changes in analysis['color_changes'].items():
            if len(changes) == 1:
                src, dest = next(iter(changes))
                if src not in color_map:
                    color_map[src] = dest
                elif color_map[src] != dest:
                    return None
        
        if not color_map:
            return None
            
        cases = "\n".join([f"    if x=={int(src)}: return {int(dest)}" for src, dest in color_map.items()])
        return f"""def p(g):
 return[[(lambda x:
{cases}
    else x)(x)for x in r]for r in g]
"""
    
    def handle_grid_operations(self, task_data, analysis):
        if not analysis['symmetry']:
            return None
        sym_type = analysis['symmetry']
        if sym_type == 'rotate_90':
            return """def p(g):
 return[list(r)for r in zip(*g[::-1])]
"""
        if sym_type == 'rotate_180':
            return """def p(g):
 return[r[::-1]for r in g[::-1]]
"""
        if sym_type == 'rotate_270':
            return """def p(g):
 return[list(r)for r in zip(*g)][::-1]
"""
        if sym_type == 'flip_vertical':
            return """def p(g):
 return g[::-1]
"""
        if sym_type == 'flip_horizontal':
            return """def p(g):
 return[r[::-1]for r in g]
"""
        return None

    def handle_resize(self, task_data, analysis):
        """✅ NEW: Handle simple resizing (cropping/padding)"""
        first_pair = task_data['train'][0]
        in_grid = np.array(first_pair['input'])
        out_grid = np.array(first_pair['output'])
        h_out, w_out = out_grid.shape
        h_in, w_in = in_grid.shape
        
        if (h_out < h_in) or (w_out < w_in):  # Crop
            return f"""def p(g):
 return [row[:{w_out}] for row in g[:{h_out}]]
"""
        if (h_out > h_in) or (w_out > w_in):  # Pad
            return f"""def p(g):
 return [row+[0]*({w_out}-len(row)) for row in g]+[[0]*{w_out} for _ in range({h_out}-len(g))]
"""
        return None

    def handle_object_operations(self, task_data, analysis):
        obj_changes = [obj for example in analysis['object_properties'] for obj in example]
        if not obj_changes:
            return None
        if len(obj_changes) == 1 and isinstance(obj_changes[0]['position_change'], np.ndarray):
            dy, dx = map(int, obj_changes[0]['position_change'])
            return f"""def p(g):
 h,w=len(g),len(g[0])
 return[[g[i-{dy}][j-{dx}] if 0<=i-{dy}<h and 0<=j-{dx}<w else 0 for j in range(w)]for i in range(h)]
"""
        return None
    
    def handle_pattern_replication(self, task_data, analysis):
        first_pair = task_data['train'][0]
        in_grid = np.array(first_pair['input'])
        out_grid = np.array(first_pair['output'])
        if (out_grid.shape[0] % in_grid.shape[0] == 0 and 
            out_grid.shape[1] % in_grid.shape[1] == 0):
            tiles_y = out_grid.shape[0] // in_grid.shape[0]
            tiles_x = out_grid.shape[1] // in_grid.shape[1]
            return f"""def p(g):
 return[[g[i%{in_grid.shape[0]}][j%{in_grid.shape[1]}]for j in range(len(g[0])*{tiles_x})]for i in range(len(g)*{tiles_y})]
"""
        return None
    
    def handle_mirror_symmetry(self, task_data, analysis):
        first_pair = task_data['train'][0]
        in_grid = np.array(first_pair['input'])
        out_grid = np.array(first_pair['output'])
        if np.array_equal(out_grid, in_grid[:, ::-1]) and in_grid.shape == out_grid.shape:
            return """def p(g):
 return[r[::-1]for r in g]
"""
        return None
    
    def handle_arithmetic_operations(self, task_data, analysis):
        if not analysis['arithmetic']:
            return None
        op, val = analysis['arithmetic']
        if op == 'add':
            return f"""def p(g):
 return[[x+{val}for x in r]for r in g]
"""
        if op == 'multiply':
            return f"""def p(g):
 return[[x*{val}for x in r]for r in g]
"""
        return None
    
    def handle_conditional_operations(self, task_data, analysis):
        first_pair = task_data['train'][0]
        in_grid = np.array(first_pair['input'])
        out_grid = np.array(first_pair['output'])
        if in_grid.shape != out_grid.shape:
            return None
        diff = out_grid - in_grid
        changed = np.where(diff != 0)
        for i,j in zip(*changed):
            if i > 0 and in_grid[i-1,j] == out_grid[i,j]:
                return """def p(g):
 return[[g[i-1][j]if i>0 and g[i][j]!=0 else g[i][j]for j in range(len(g[0]))]for i in range(len(g))]
"""
            if j > 0 and in_grid[i,j-1] == out_grid[i,j]:
                return """def p(g):
 return[[g[i][j-1]if j>0 and g[i][j]!=0 else g[i][j]for j in range(len(g[0]))]for i in range(len(g))]
"""
        return None
    
    def verify_solution(self, solution_code, task_data):
        try:
            exec(solution_code, globals())
            all_examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
            for example in all_examples[:3]:
                input_grid = example['input']
                expected = example['output']
                actual = p(input_grid)
                if actual != expected:
                    return False
            return True
        except:
            return False

def create_arc_solutions():
    generator = ARCSolutionGenerator()
    solutions = {}
    for task_num in range(1, 401):
        task_id = f"{task_num:03d}"
        try:
            with open(f"/kaggle/input/google-code-golf-2025/task{task_id}.json") as f:
                task_data = json.load(f)
            solution = generator.generate_solution(task_data)
            solutions[task_id] = solution
            if generator.verify_solution(solution, task_data):
                print(f"Task {task_id} ✅")
            else:
                print(f"Task {task_id} ❌ (used fallback)")
        except Exception as e:
            print(f"Error processing task {task_id}: {str(e)}")
            solutions[task_id] = """def p(g):
 return [row[:] for row in g]
"""
    os.makedirs("/kaggle/working/submission", exist_ok=True)
    for task_id, code in solutions.items():
        with open(f"/kaggle/working/submission/task{task_id}.py", "w") as f:
            f.write(code)
    with zipfile.ZipFile("/kaggle/working/submission.zip", "w") as zipf:
        for task_id in solutions:
            zipf.write(f"/kaggle/working/submission/task{task_id}.py", 
                       f"task{task_id}.py")
    print("\nEnhanced ARC solutions created: submission.zip")

create_arc_solutions()
